In [11]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Baixando os recursos necessários do NLTK
nltk.download('stopwords')

# Carregando os dados
try:
    # Atualize para o caminho local do arquivo CSV se necessário
    url = "/content/twitter_training.csv"

    # Especificar os nomes das colunas se o CSV não tiver cabeçalhos
    column_names = ['id', 'game', 'sentiment', 'text']
    df = pd.read_csv(url, header=None, names=column_names)

    print("Dados carregados com sucesso.")
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    df = None  # Definindo df como None para evitar erros posteriores

# Verifique se df foi carregado corretamente antes de prosseguir
if df is not None:
    # Verifique os nomes das colunas
    print("\nNomes das colunas:")
    print(df.columns)

    # Explorando os dados
    print("\nAmostra dos dados:")
    print(df.head())

    # Verifique se a coluna 'sentiment' existe
    if 'sentiment' in df.columns:
        print("\nDistribuição das classes de sentimento:")
        print(df['sentiment'].value_counts())
    else:
        print("\nErro: A coluna 'sentiment' não foi encontrada no DataFrame.")

    # Pré-processamento dos dados
    stop_words = stopwords.words('english')

    def preprocess_text(text):
        if isinstance(text, str):
            text = text.lower()
            text = ' '.join([word for word in text.split() if word not in stop_words])
            return text
        else:
            return ''

    # Tratar valores nulos e garantir que todos os valores sejam strings
    df['text'] = df['text'].fillna('').astype(str)
    df['clean_text'] = df['text'].apply(preprocess_text)

    # Separação dos dados em treino e teste
    X = df['clean_text']
    y = df['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Treinando o modelo de PLN, utilizando Naive Bayes
    model = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', MultinomialNB())
    ])

    model.fit(X_train, y_train)

    # Avaliação do modelo
    y_pred = model.predict(X_test)
    print("\nRelatório de Classificação:")
    print(metrics.classification_report(y_test, y_pred))

    # Realizando testes com frases de exemplo
    exemplo_frases = [
        "I love this product!",
        "This is the worst experience I've ever had.",
        "It's okay, not bad.",
        "Absolutely fantastic!",
        "I hate it so much.",
    ]

    predicoes = model.predict(exemplo_frases)
    for frase, sentimento in zip(exemplo_frases, predicoes):
        print(f'Frase: "{frase}"\nSentimento Predito: {sentimento}\n')

    # Análise crítica do modelo
    print("\nAnálise Crítica do Modelo:")
    print("Pontos fortes: O modelo foi capaz de capturar bem as polaridades extremas, como 'amor' e 'ódio'.")
    print("Pontos fracos: O modelo pode ter dificuldade em detectar sentimentos mais sutis ou ambíguos.")
    print("Oportunidades de melhorias: Poderíamos experimentar outros modelos como SVM ou redes neurais, e ajustar os hiperparâmetros.")
    print("Frases em que o modelo falhou: Nenhum exemplo específico foi identificado neste teste, mas frases neutras ou sarcásticas poderiam ser problemáticas.")

    # Informações adicionais
    print("\nInformações adicionais:")
    print("Os dados foram divididos em 80% para treino e 20% para teste.")
    print("O pré-processamento incluiu a remoção de stopwords e conversão para minúsculas.")
    print("O modelo utilizado foi Naive Bayes, treinado com TF-IDF.")
    print("O modelo suporta análises em inglês.")
else:
    print("Não foi possível carregar os dados. Verifique o caminho do arquivo e tente novamente.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Dados carregados com sucesso.

Nomes das colunas:
Index(['id', 'game', 'sentiment', 'text'], dtype='object')

Amostra dos dados:
     id         game sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                text  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  

Distribuição das classes de sentimento:
sentiment
Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

Relatório de Classificação:
              precision    recall  f1-score   support

  Irrelevant       0.96      0.43      0.59      3922
    Negative       0.65      0.90      0.